In [24]:
!pip install nltk
!pip install pyspellchecker

     ---------------------------------------- 2.5/2.5 MB 589.1 kB/s eta 0:00:00


In [176]:
DOC_NUM=1400

In [ ]:
import string
import nltk
import os
import math
import numpy as np

# Define the corpus
corpus=[]
for i in range(1,1400):
    with open(f"./Cranfield/{i}.txt", "r") as f:
        corpus.append(f.read())

# Tokenize the documents and remove stopwords and punctuation
stopwords = nltk.corpus.stopwords.words("english")
stemmer = nltk.stem.PorterStemmer()

def preprocess(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [stemmer.stem(token) for token in tokens if token not in stopwords and token not in string.punctuation]
    return tokens

tokenized_corpus = [preprocess(text) for text in corpus]

# Build the document-term matrix
terms = set([token for doc in tokenized_corpus for token in doc])
term_index = {term: i for i, term in enumerate(sorted(terms))}
doc_term_matrix = np.zeros((len(tokenized_corpus), len(terms)))

for i, doc in enumerate(tokenized_corpus):
    for token in doc:
        j = term_index[token]
        doc_term_matrix[i, j] += 1

# Calculate the IDF values
doc_freq = np.sum(doc_term_matrix > 0, axis=0)
idf = np.log(len(tokenized_corpus) / doc_freq)

# Calculate the TF-IDF matrix
tfidf = np.zeros_like(doc_term_matrix, dtype=float)
for i in range(len(tokenized_corpus)):
    tfidf[i] = doc_term_matrix[i] * idf




In [203]:
with open(f'./RES/{1}.txt','r') as f:
    print(f.readlines()[2])

1 31	2



In [222]:
# Add score for each queries
true_answer={}
for i in range(1,225+1):
    with open(f'./RES/{i}.txt','r') as f:
        true_answer[i]={}
        for line in f.readlines():
            temp=line.replace(" ","\t").split()
            if(len(temp)<3):
                continue
            _,index,points=temp
            index,points=int(index),int(points)
            if(points==-1):
                true_answer[i][index]=1
            elif points==1:
                true_answer[i][index]=0.8
            elif points==2:
                true_answer[i][index]=0.7
            elif points==3:
                true_answer[i][index]=0.6
            elif points==4:
                true_answer[i][index]=0.5

In [225]:
# Add queries from folder
queries=[]
queries.append("")
with open(f'./query.txt','r') as f:
    for line in f.readlines():
        queries.append(line[2:-1])

In [228]:

scores={0:0}
for u,query in enumerate(queries[1:]):
    i=u+1
    scores[i]=0
    # Search for documents that contain a query term
    query_tokens = preprocess(query)
    query_vector = np.zeros(len(terms))
    if(len(query_tokens)==0):
        continue
    for token in query_tokens:
        if token in term_index:
            j = term_index[token]
            query_vector[j] += 1

    query_tfidf = query_vector * idf
    
    similarity_scores = np.dot(tfidf, query_tfidf) / (np.linalg.norm(tfidf, axis=1) * np.linalg.norm(query_tfidf))

    # Rank the documents by their similarity scores
    ranking = similarity_scores.argsort()[-20:][::-1]+1

    for rank in ranking:
        scores[i]+=true_answer[i].get(rank,0)

C:\Users\Jakiro\AppData\Local\Temp\ipykernel_12792\894788197.py:16: RuntimeWarning: invalid value encountered in divide
  similarity_scores = np.dot(tfidf, query_tfidf) / (np.linalg.norm(tfidf, axis=1) * np.linalg.norm(query_tfidf))


In [229]:
scores

{0: 0,
 1: 5.1,
 2: 4.0,
 3: 5.199999999999999,
 4: 2.2,
 5: 2.2,
 6: 2.3,
 7: 2.2,
 8: 4.6,
 9: 0,
 10: 2.9,
 11: 2.6,
 12: 2.9000000000000004,
 13: 1,
 14: 1.3,
 15: 1.4,
 16: 1.6,
 17: 1.7,
 18: 1.7,
 19: 0,
 20: 5.799999999999999,
 21: 3.2,
 22: 0,
 23: 5.800000000000001,
 24: 2.4,
 25: 4.3,
 26: 2.3,
 27: 1.7,
 28: 0,
 29: 3.0999999999999996,
 30: 1,
 31: 1,
 32: 1,
 33: 2.1,
 34: 4.2,
 35: 0,
 36: 0.6,
 37: 4.2,
 38: 1.6,
 39: 3.2,
 40: 2.2,
 41: 2.3,
 42: 2.9,
 43: 3.6000000000000005,
 44: 0,
 45: 3.0,
 46: 3.8000000000000003,
 47: 5.7,
 48: 2.3,
 49: 1.6,
 50: 0.7,
 51: 4.8,
 52: 3.5999999999999996,
 53: 1.6,
 54: 1.7,
 55: 3.0999999999999996,
 56: 2.6,
 57: 2.1,
 58: 1,
 59: 1.2,
 60: 2.7,
 61: 3.0,
 62: 0.6,
 63: 0,
 64: 2.5,
 65: 4.8999999999999995,
 66: 2.0,
 67: 6.699999999999999,
 68: 1.7,
 69: 1,
 70: 1.7,
 71: 1.8,
 72: 2.3,
 73: 5.3999999999999995,
 74: 1.2,
 75: 0.6,
 76: 2.9,
 77: 4.5,
 78: 2.9000000000000004,
 79: 1,
 80: 1,
 81: 2.3,
 82: 1.2999999999999998,
 83: 0

In [ ]:
query = "what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft  "
query_tokens = preprocess(query)
query_vector = np.zeros(len(terms))

for token in query_tokens:
    if token in term_index:
        j = term_index[token]
        query_vector[j] += 1

query_tfidf = query_vector * idf
similarity_scores = np.dot(tfidf, query_tfidf) / (np.linalg.norm(tfidf, axis=1) * np.linalg.norm(query_tfidf))

# Rank the documents by their similarity scores
ranking = similarity_scores.argsort()[-20:][::-1]

for i in ranking:
    print(f"Document {i+1}: {corpus[i]}")